In [1]:
import numpy as np

In [2]:
'''
type            : type of objective fun
No_of_variable  :  totol num of variable
Type_of_variable : contain xi >= , <= or unrestricted
No_of_constraints :total num of constrians
sign_of_constraints : constrains are <= , >= , = 
c : coeff. of objective fun.
A : coeff. of constrains
b : right side of constrains
'''

def convert_to_standard_form(
    Type , No_of_variable , Type_of_variable ,
    No_of_constraints , sign_of_constraints ,
    c , A , b ):
      
    if Type == 'min':
        Type = "max"
        c = [(-1)*ele for ele in c]
    
    i1 = 0
    while i1 < No_of_constraints:
         # convert >= to change <= there for multiple with (-1) sign
        if sign_of_constraints[i1] == ">=":
            sign_of_constraints[i1] = "<="
            A[i1] = (-1)*A[i1]
            b[i1] = (-1)*b[i1]
            
        # convert = constrain to <= and >=   
        elif sign_of_constraints[i1] == "=":
            sign_of_constraints[i1] = "<="
            No_of_constraints += 1
            sign_of_constraints.append("<=")
            A = np.vstack((A,(-1)*A[i1]))
            b.append((-1)*b[i1])
            
        # <= then just pass 
        else:
            pass
        
        i1 += 1    
    i2 = 0
    

    while i2 < No_of_variable:
        # if type of variable are xi <=0 then we have to change to xi >= 0 
        if Type_of_variable[i2] == "<=":
            
            Type_of_variable[i2] = ">="
            for j in range(No_of_constraints):
                A[j][i2] = (-1)*A[j][i2]
            c[i2] = (-1)*c[i2]
        
        # if xi is unrestricted then xi = xi`-xi``
        elif Type_of_variable[i2] == "unrestricted":
            Type_of_variable[i2] = ">="
            No_of_variable += 1
            Type_of_variable.append(">=")
            new_col = []
            for j in range(No_of_constraints):
                new_col.append((-1)*A[j][i2])
                
            # concate A and updated type of variable
            A = np.hstack((A, np.atleast_2d(new_col).T))
            c.append((-1)*c[i2])
        
        
        else:
            pass
        
        i2 += 1
    #return updated all inputs so we can use further 
    return Type,No_of_variable,Type_of_variable,No_of_constraints,sign_of_constraints,c,A,b

In [3]:
# convert to dual

def convert_to_Dual(
    Type , No_of_variable , Type_of_variable ,
    No_of_constraints , sign_of_constraints ,
    c , A , b ):
    
    Dual_Type = "min"
    No_of_variable_of_dual = No_of_constraints
    Type_of_variable_of_dual = []
    
    # change inequality in dual
    for i in range(No_of_constraints):
        if sign_of_constraints[i]=="<=":
            Type_of_variable_of_dual.append(">=")
        else:
            return print("Please first convert to standard form!")
    
    No_of_constraints_of_dual = No_of_variable
    sign_of_constraints_of_dual = []
    for i in range(No_of_variable):
        if Type_of_variable[i]==">=":
            sign_of_constraints_of_dual.append(">=")
            
    Dual_c = b
    Dual_A = A.T
    Dual_b = c
    
    return Dual_Type,No_of_variable_of_dual,Type_of_variable_of_dual,No_of_constraints_of_dual,sign_of_constraints_of_dual,Dual_c,Dual_A,Dual_b

In [4]:
def print_LPP(
    P_or_D , Type,No_of_variable , Type_of_variable ,
    No_of_constraints , sign_of_constraints ,
    c , A , b):
    
    if P_or_D == 'Primal':
        z = ""
        
        for i in range(No_of_variable):
            z = z + "{}*x{} + ".format(c[i],(i+1))
        z = z[:-2]
        print(Type,"z =",z)
        print()
        print("Subject to Constraints")
        
        for i in range(No_of_constraints):
            raw = ""
            for j in range(No_of_variable):
                raw = raw + "{}*x{} + ".format(A[i][j],j+1)
            raw = raw[:-2]
            raw = raw + " {} ".format(sign_of_constraints[i]) + " {}".format(b[i])
            print(raw)
            
        print()
        
        for i in range(No_of_variable):
            x = "x{} ".format(i+1) + "{}".format(Type_of_variable[i])
            if Type_of_variable[i]!="unrestricted":
                x += " 0"
            print(x)
            
    else:
        
        w = ""
        
        for i in range(No_of_variable):
              w = w + "{}*y{} + ".format(c[i],(i+1))
        w = w[:-2]
        print(Type,"w =",w)
        
        print()
        
        print("Subject to Constraints")
        for i in range(No_of_constraints):
            raw = ""
            for j in range(No_of_variable):
                raw = raw + "{}*y{} + ".format(A[i][j],j+1)
            raw = raw[:-2]
            raw = raw + " {} ".format(sign_of_constraints[i]) + " {}".format(b[i])
            print(raw)
            
        print()
        
        for i in range(No_of_variable):
            y = "y{} ".format(i+1) + "{}".format(Type_of_variable[i])
            if Type_of_variable[i]!="unrestricted":
                y += " 0"
            print(y)
            
        print()
        
        Unrestricted_variables = []
        n = No_of_variable
        k = 0
        
        while k < n:
            if (-c[k]) in c:
                Unrestricted_variables.append(k)
                Unrestricted_variables.append(c.index(-c[k]))
                n -= 1
            k += 1
            
        Unrestricted_variables = [str(ele+1) for ele in Unrestricted_variables]
        print("Here {} are make total {} unrestricted variables and other are Non-Negative variable.".format(','.join(Unrestricted_variables),int(len(Unrestricted_variables)/2)))

In [5]:
# for which type of objective function
Type = input("Enter a type of objective function whether its max or min : ")

# for how may xi's are present
No_of_variable = int(input("Enter number of variable : "))

# for xi <= , >= , unrestricted
print("If xi >= ,<= , unrestricted then enter >= , <= and unrestricted respectivly. ")

Type_of_variable = []
for i in range(No_of_variable):
    Type_of_variable.append(input("Enter the sign of variable {} :".format(i+1)))
print()

#for number of constraints

c = []
for i in range(No_of_variable):
    c.append(float(input("Enter the value c {} : ".format(i+1))))
No_of_constraints = int(input("Enter number of constraints : "))

#sign of constraints

sign_of_constraints = []

for i in range(No_of_constraints):
    sign_of_constraints.append(input("Enter the sign of constrint {} : ".format(i+1)))
b = []

print()

print("Enter Right side of constraints")
for i in range(No_of_constraints):
    b.append(float(input("Enter value of b(right side) of constraint {} : ".format(i+1))))

print()

print("Enter coefficient of variables")
A = []

for i in range(No_of_constraints):
    row = []
    for j in range(No_of_variable):
        row.append(float(input("Enter the value of coefficient of variable {} of row {}: ".format(j+1,i+1))))
    A.append(row)
    
A = np.array(A)

Enter a type of objective function whether its max or min : max
Enter number of variable : 3
If xi >= ,<= , unrestricted then enter >= , <= and unrestricted respectivly. 
Enter the sign of variable 1 :<=
Enter the sign of variable 2 :>=
Enter the sign of variable 3 :unrestricted

Enter the value c 1 : 4
Enter the value c 2 : 6
Enter the value c 3 : 10
Enter number of constraints : 3
Enter the sign of constrint 1 : <=
Enter the sign of constrint 2 : >=
Enter the sign of constrint 3 : =

Enter Right side of constraints
Enter value of b(right side) of constraint 1 : 7
Enter value of b(right side) of constraint 2 : 3
Enter value of b(right side) of constraint 3 : 7

Enter coefficient of variables
Enter the value of coefficient of variable 1 of row 1: 2
Enter the value of coefficient of variable 2 of row 1: 5
Enter the value of coefficient of variable 3 of row 1: 6
Enter the value of coefficient of variable 1 of row 2: 24
Enter the value of coefficient of variable 2 of row 2: 5
Enter the va

In [6]:
A0 = A.copy()
b0 = b.copy()
c0 = c.copy()

n = No_of_variable
m = No_of_constraints

T_v = Type_of_variable.copy()
s_c = sign_of_constraints.copy()

T = Type

In [7]:
print_LPP("Primal",T,n,T_v,m,s_c,c0,A0,b0)

max z = 4.0*x1 + 6.0*x2 + 10.0*x3 

Subject to Constraints
2.0*x1 + 5.0*x2 + 6.0*x3  <=  7.0
24.0*x1 + 5.0*x2 + 9.0*x3  >=  3.0
11.0*x1 + 4.0*x2 + 8.0*x3  =  7.0

x1 <= 0
x2 >= 0
x3 unrestricted


In [8]:
new_T,new_n,new_T_v,new_m,new_s_c,new_c0,new_A0,new_b0 = convert_to_standard_form(T,n,T_v,m,s_c,c0,A0,b0)

In [9]:
print_LPP("Primal",new_T,new_n,new_T_v,new_m,new_s_c,new_c0,new_A0,new_b0)

max z = -4.0*x1 + 6.0*x2 + 10.0*x3 + -10.0*x4 

Subject to Constraints
-2.0*x1 + 5.0*x2 + 6.0*x3 + -6.0*x4  <=  7.0
24.0*x1 + -5.0*x2 + -9.0*x3 + 9.0*x4  <=  -3.0
-11.0*x1 + 4.0*x2 + 8.0*x3 + -8.0*x4  <=  7.0
11.0*x1 + -4.0*x2 + -8.0*x3 + 8.0*x4  <=  -7.0

x1 >= 0
x2 >= 0
x3 >= 0
x4 >= 0


In [10]:
Dual_T,Dual_n,Dual_T_v,Dual_m,Dual_s_c,Dual_c0,Dual_A0,Dual_b0 = convert_to_Dual(new_T,new_n,new_T_v,new_m,new_s_c,new_c0,new_A0,new_b0)

In [11]:
print_LPP("Dual",Dual_T,Dual_n,Dual_T_v,Dual_m,Dual_s_c,Dual_c0,Dual_A0,Dual_b0)

min w = 7.0*y1 + -3.0*y2 + 7.0*y3 + -7.0*y4 

Subject to Constraints
-2.0*y1 + 24.0*y2 + -11.0*y3 + 11.0*y4  >=  -4.0
5.0*y1 + -5.0*y2 + 4.0*y3 + -4.0*y4  >=  6.0
6.0*y1 + -9.0*y2 + 8.0*y3 + -8.0*y4  >=  10.0
-6.0*y1 + 9.0*y2 + -8.0*y3 + 8.0*y4  >=  -10.0

y1 >= 0
y2 >= 0
y3 >= 0
y4 >= 0

Here 1,4,3,4 are make total 2 unrestricted variables and other are Non-Negative variable.
